# 例子 - 使用 `huggingface/transfomers.GPT2LMHeadModel` 进行微调

> ❗ **要点**:
>
> NVIDIA/Megatron-LM 的 `tokenizer` 会在 `128` 对齐的基础上，强制加上 `8` 个特殊 `token`，所以注意 `token` 的偏移！


该 Notebook 的代码大部分来自: <https://github.com/huggingface/transformers/blob/master/examples/run_generation.py>

## 切换工作目录

注意不要搞错!按照实际情况切换！

In [1]:
%cd ..

/home/Public/Megatron-LM


## 环境变量

- `CUDA` 设备:

In [2]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


## 运行

In [4]:
%run finetune_gpt2.py \
    --model_name_or_path checkpoints/hfgpt2-117m-emotion/ \
    --output_dir checkpoints/tr_191023 \
    --overwrite_output_dir \
    --tokenizer_model_path data/spm/gpt2_huamei_corpus_bpe_32k_v2.model \
    --do_train \
    --train_data_file data/tr_191023.train.json \
    --do_eval \
    --eval_data_file data/tr_191023.val.json \
    --fp16 \
    --num_train_epochs 1 \
    --logging_steps 100 \
    --save_steps 1000 \
    --per_gpu_train_batch_size 2

2019-10-23 18:19:00,979 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: True
2019-10-23 18:19:00,980 - INFO - transformers.configuration_utils -   loading configuration file checkpoints/hfgpt2-117m-emotion/config.json
2019-10-23 18:19:00,981 - INFO - transformers.configuration_utils -   Model config {
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "finetuning_task": null,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 1024,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torchscript": false,
  "use_bfloat16": false,
  "vocab_size": 32128
}

2019-10-23 18:19:01,055 - INFO - t

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


/home/liuxy/miniconda3/envs/Megatron_LM-ipy/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:73: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


KeyboardInterrupt: 